# Phase 2 : Analyse exploratoire des données (EDA)

Ce notebook présente le pipeline complet d'analyse exploratoire pour la détection de fraudes bancaires.

**Objectifs :**
- Charger les données de manière robuste (CSV/Parquet)
- Explorer et valider la qualité des données
- Réaliser au moins 15 visualisations interactives avec Plotly
- Effectuer des tests statistiques pour valider les observations
- Préparer les données pour la modélisation

---

## 1. Importation des bibliothèques nécessaires

Nous allons utiliser pandas, numpy, plotly, scipy, et le module data_loader.py pour charger et analyser les données.

In [ ]:
# Importation des librairies
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
from src.data.data_loader import DataLoader

## 2. Chargement des données avec data_loader.py

Utilisation du module DataLoader pour charger les données bancaires au format CSV ou Parquet.

In [ ]:
# Chargement des données
loader = DataLoader('config/config.yaml')
data = loader.load_data()
print(f"Dimensions des données : {data.shape}")
data.head()

In [ ]:
## 3. Aperçu et validation des données

Affichage des premières lignes, types de données, valeurs manquantes et statistiques descriptives.

## 3. Analyse du Déséquilibre des Classes {#imbalance}

In [ ]:
# Aperçu des données
print(data.info())
print(data.describe())
print("Valeurs manquantes par colonne :")
print(data.isnull().sum())

## 4. Nettoyage et préparation des données

Traitement des valeurs manquantes, des doublons et conversion des types si nécessaire.

In [ ]:
# Nettoyage des données
# Suppression des doublons
data = data.drop_duplicates()
# Imputation simple des valeurs manquantes (si besoin)
data = data.fillna(data.median(numeric_only=True))
# Conversion des types si nécessaire
for col in ['Time', 'Amount', 'Class']:
    data[col] = pd.to_numeric(data[col], errors='coerce')
print("Nettoyage terminé. Dimensions :", data.shape)

## 5. Visualisations exploratoires

Création d'au moins 15 visualisations pour explorer les distributions et relations entre variables.

In [ ]:
# Visualisation 1 : Histogramme du montant
px.histogram(data, x='Amount', nbins=50, title='Distribution des montants')

# Visualisation 2 : Histogramme du temps
px.histogram(data, x='Time', nbins=50, title='Distribution du temps')

# Visualisation 3 : Boxplot du montant par classe
px.box(data, x='Class', y='Amount', title='Montant par classe (fraude/non-fraude)')

# Visualisation 4 : Bar chart du nombre de transactions par classe
px.bar(data['Class'].value_counts().reset_index(), x='index', y='Class', title='Répartition des classes')

# Visualisation 5 : Pie chart de la proportion de fraudes
px.pie(data, names='Class', title='Proportion de fraudes')

### Visualisations complémentaires

Exploration des relations entre variables et détection d'outliers.

In [ ]:
# Visualisation 6 : Scatter plot Amount vs Time
px.scatter(data, x='Time', y='Amount', color='Class', title='Montant vs Temps selon la classe')

# Visualisation 7 : Heatmap de corrélation
import plotly.figure_factory as ff
corr = data.corr()
fig = ff.create_annotated_heatmap(z=corr.values, x=list(corr.columns), y=list(corr.index), colorscale='Viridis')
fig.update_layout(title='Matrice de corrélation')
fig.show()

# Visualisation 8 : Violin plot du montant par classe
px.violin(data, y='Amount', x='Class', box=True, points='all', title='Violin plot du montant par classe')

# Visualisation 9 : KDE plot du montant
import plotly.figure_factory as ff
fig = ff.create_distplot([data[data['Class']==0]['Amount'], data[data['Class']==1]['Amount']], group_labels=['Non-Fraude', 'Fraude'], show_hist=False)
fig.update_layout(title='KDE du montant par classe')
fig.show()

# Visualisation 10 : Outliers sur le montant
q1 = data['Amount'].quantile(0.25)
q3 = data['Amount'].quantile(0.75)
iqr = q3 - q1
outliers = data[(data['Amount'] < q1 - 1.5*iqr) | (data['Amount'] > q3 + 1.5*iqr)]
px.scatter(outliers, x='Time', y='Amount', color='Class', title='Outliers sur le montant')

### Visualisations avancées et analyse du déséquilibre

Analyse des variables, déséquilibre des classes et exploration des patterns de fraude.

In [ ]:
# Visualisation 11 : Montant moyen par heure
if 'Time' in data.columns:
    data['Hour'] = (data['Time'] // 3600).astype(int)
    px.bar(data.groupby('Hour')['Amount'].mean().reset_index(), x='Hour', y='Amount', title='Montant moyen par heure')

# Visualisation 12 : Nombre de transactions par jour (si colonne Date existe)
if 'Date' in data.columns:
    px.bar(data.groupby('Date').size().reset_index(name='Transactions'), x='Date', y='Transactions', title='Transactions par jour')

# Visualisation 13 : Distribution d'une variable aléatoire
col = data.columns[3] if len(data.columns) > 3 else 'Amount'
px.histogram(data, x=col, nbins=50, title=f'Distribution de {col}')

# Visualisation 14 : Boxplot par variable
for col in ['Amount', 'Time']:
    px.box(data, y=col, title=f'Boxplot de {col}')

# Visualisation 15 : Heatmap des valeurs manquantes
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
sns.heatmap(data.isnull(), cbar=False)
plt.title('Heatmap des valeurs manquantes')
plt.show()

## 6. Tests statistiques sur les observations

Réalisation de tests statistiques pour valider les observations issues de l'EDA.

In [ ]:
# Test de normalité (Shapiro-Wilk)
from scipy.stats import shapiro
stat, p = shapiro(data['Amount'])
print(f"Test de Shapiro-Wilk sur 'Amount': Stat={stat:.3f}, p-value={p:.3f}")

# Test de corrélation (Pearson)
from scipy.stats import pearsonr
corr, p_corr = pearsonr(data['Amount'], data['Time'])
print(f"Corrélation Pearson Amount/Time: r={corr:.3f}, p-value={p_corr:.3f}")

# Test de comparaison de moyennes (t-test)
from scipy.stats import ttest_ind
amount_fraud = data[data['Class']==1]['Amount']
amount_nonfraud = data[data['Class']==0]['Amount']
t_stat, t_p = ttest_ind(amount_fraud, amount_nonfraud, equal_var=False)
print(f"T-test Amount (Fraude vs Non-Fraude): Stat={t_stat:.3f}, p-value={t_p:.3f}")

# Test du chi2 sur la répartition des classes
from scipy.stats import chi2_contingency
table = pd.crosstab(data['Class'], data['Amount'] > data['Amount'].median())
chi2, chi2_p, _, _ = chi2_contingency(table)
print(f"Test du chi2 sur la répartition des classes: Stat={chi2:.3f}, p-value={chi2_p:.3f}")

## 7. Visualisations interactives avec Plotly

Certaines visualisations ci-dessus sont interactives grâce à Plotly. Utilisez les outils d'exploration pour zoomer, filtrer et analyser les patterns de fraude.